# SLURM full aggregated data analysis

This notebook will run statistic information on the whole aggregated slurm dataset jobs to gain insights on the job system

In [7]:
import pandas as pd
import numpy as np

In [8]:
slurm = pd.read_csv("/mnt/scratch/alkhali7/DID_FINAL_SLURM_OCT_2023_aggregated.csv", delimiter=",")

/tmp/local/33543227/ipykernel_17342/1776628446.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  slurm = pd.read_csv("/mnt/scratch/alkhali7/DID_FINAL_SLURM_OCT_2023_aggregated.csv", delimiter=",")


In [9]:
slurm.head()

,Unnamed: 0,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
0,0,31496544,user_679,group_121,2023-03-21T11:13:45,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,28,0,21000M,NaN,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",NaN
1,1,31497932,user_679,group_121,2023-03-21T11:31:18,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,28,0,21000M,NaN,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",NaN
2,2,31993628,user_105,group_114,2023-03-22T18:19:12,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,12,0,150G,NaN,1,1,None assigned,"billing=23347,cpu=12,gres/gpu=8,mem=150G,node=1",NaN
3,3,39087660,user_652,group_054,2023-04-04T13:09:10,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,640,0,20G,NaN,10,10,None assigned,"billing=3112,cpu=640,mem=20G,node=10",NaN
4,4,59062820,user_188,group_046,2023-05-08T09:58:20,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [26]:
# slurm.info()
print(slurm.columns)
print(slurm.shape)

Index(['Unnamed: 0', 'JobID', 'User', 'Group', 'Submit', 'Start', 'End',
       'Elapsed', 'State', 'Account', 'AssocID', 'Partition', 'Timelimit',
       'UserCPU', 'SystemCPU', 'TotalCPU', 'CPUTime', 'CPUTimeRAW', 'ReqCPUS',
       'AllocCPUS', 'ReqMem', 'MaxRSS', 'ReqNodes', 'NNodes', 'NodeList',
       'ReqTRES', 'AllocTRES'],
      dtype='object')
(3364285, 27)


In [27]:
slurm.Timelimit

4          2-00:00:00
5          2-00:00:00
6          2-00:00:00
7          2-00:00:00
8          2-00:00:00
              ...    
3369217      01:00:00
3369218      01:00:00
3369219      00:10:00
3369220      00:10:00
3369221      00:10:00
Name: Timelimit, Length: 3364285, dtype: object

In [30]:
slurm.ReqMem

4               8G
5               8G
6               8G
7               8G
8               8G
            ...   
3369217    505202M
3369218    505202M
3369219        20G
3369220        20G
3369221        20G
Name: ReqMem, Length: 3364285, dtype: object

# Preprocessing

In [22]:
# # Filter out rows where 'State' is "Cancelled" or Unknown
slurm = slurm[slurm['State'] != 'Cancelled']
slurm = slurm[slurm['Start']!= 'Unknown']

slurm['Submit'] = pd.to_datetime(slurm['Submit'])
slurm['Start'] = pd.to_datetime(slurm['Start'])
slurm['End'] = pd.to_datetime(slurm['Start'])

In [23]:
slurm.head()

,Unnamed: 0,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
4,4,59062820,user_188,group_046,2023-05-08 09:58:20,2024-01-01 00:58:57,2024-01-01 00:58:57,00:00:09,COMPLETED,account_017,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
5,5,59062828,user_188,group_046,2023-05-08 09:58:20,2024-01-01 01:00:49,2024-01-01 01:00:49,00:00:06,COMPLETED,account_017,...,240,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
6,6,59062836,user_188,group_046,2023-05-08 09:58:20,2024-01-01 01:00:49,2024-01-01 01:00:49,00:00:06,COMPLETED,account_017,...,240,40,40,8G,NaN,1,1,skl-030,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
7,7,59062844,user_188,group_046,2023-05-08 09:58:21,2024-01-01 00:58:57,2024-01-01 00:58:57,00:00:09,COMPLETED,account_017,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
8,8,59062852,user_188,group_046,2023-05-08 09:58:21,2024-01-01 00:58:57,2024-01-01 00:58:57,00:00:09,COMPLETED,account_017,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [46]:
# convert ReqMem to a uniform measurement to MB (assuming 'M' for MB and 'G' for GB and 'K' for KB)
def convert_memory(mem_str):
    '''
    Convert memory units to MegaBytes unit float.
    '''
    if isinstance(mem_str, (int, float, complex)):   # if its a zero
        pass
    elif mem_str.endswith('M'):
        return float(mem_str[:-1]) # remove 'M' and convert to float
    elif mem_str.endswith('K'):
        return float(mem_str[:-1]) / 1000
    elif mem_str.endswith('G'):
        return float(mem_str[:-1]) * 1e3  # convert MB to KB
    elif mem_str.endswith('T'):
        return float(mem_str[:-1]) * 1e6 # convert MB to T


In [48]:
# import warnings
# warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# First drop rows with NA requested memory
slurm= slurm.dropna(subset=['ReqMem'])
slurm= slurm.dropna(subset=['MaxRSS'])


slurm['ReqMem_MB'] = slurm['ReqMem'].apply(convert_memory)
slurm['MaxRSS_MB'] = slurm['MaxRSS'].apply(convert_memory)


NameError: name 'slum' is not defined

In [49]:
slurm

,Unnamed: 0,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,...,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES,ReqMem_MB,MaxRSS_MB
201,201,87461884,user_600,group_090,2023-09-08 18:34:09,2023-09-29 21:30:06,2023-09-29 21:30:06,1-08:12:31,COMPLETED,account_011,...,64,32G,4500K,4,4,"lac-[084-086,193]","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4",32000.0,4.500
202,202,87461892,user_600,group_090,2023-09-08 18:34:11,2023-09-30 00:16:32,2023-09-30 00:16:32,1-06:03:08,COMPLETED,account_011,...,64,32G,4540K,4,4,"lac-[142,192,345,349]","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4",32000.0,4.540
203,203,87461900,user_600,group_090,2023-09-08 18:34:13,2023-09-30 01:07:56,2023-09-30 01:07:56,1-05:26:01,COMPLETED,account_011,...,64,32G,4536K,4,4,"lac-[080-081,342-343]","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4",32000.0,4.536
204,204,87461908,user_600,group_090,2023-09-08 18:34:15,2023-09-30 03:06:56,2023-09-30 03:06:56,1-06:09:06,COMPLETED,account_011,...,64,32G,4540K,4,4,"lac-[031,136,346-347]","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4",32000.0,4.540
205,205,87461916,user_600,group_090,2023-09-08 18:34:18,2023-09-30 08:13:45,2023-09-30 08:13:45,1-06:16:43,COMPLETED,account_011,...,64,32G,4604K,4,4,"lac-[083,196,344,348]","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4","billing=4980,cpu=64,gres/gpu=4,mem=32G,node=4",32000.0,4.604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3368422,3368422,99252768,user_329,group_127,2023-10-31 23:52:36,2023-10-31 23:57:13,2023-10-31 23:57:13,00:00:38,FAILED,account_011,...,16,32G,0,1,1,lac-442,"billing=4980,cpu=16,mem=32G,node=1","billing=4980,cpu=16,mem=32G,node=1",32000.0,NaN
3368600,3368600,99253488,user_103,group_062,2023-10-31 23:56:51,2023-10-31 23:56:57,2023-10-31 23:56:57,00:22:33,COMPLETED,account_017,...,4,32G,0,1,1,lac-363,"billing=4980,cpu=4,mem=32G,node=1","billing=4980,cpu=4,mem=32G,node=1",32000.0,NaN
3368651,3368651,99253828,user_060,group_154,2023-10-31 23:58:05,2023-10-31 23:58:33,2023-10-31 23:58:33,12:18:47,COMPLETED,account_017,...,2,240G,0,2,2,"amr-[250,253]","billing=37355,cpu=2,mem=240G,node=2","billing=37355,cpu=2,mem=240G,node=2",240000.0,NaN
3368698,3368698,99254020,user_060,group_154,2023-10-31 23:59:05,2023-10-31 23:59:11,2023-10-31 23:59:11,12:03:57,COMPLETED,account_017,...,2,240G,0,2,2,"amr-[197,218]","billing=37355,cpu=2,mem=240G,node=2","billing=37355,cpu=2,mem=240G,node=2",240000.0,NaN
